# Instructions
You will submit an HTML document to Canvas as your final version.  

Your document should show your code chunks/cells as well as any output. Make sure that only relevant output is printed. Do not, for example, print the entire dataset in your final rendered file.  

Your document should also be clearly organized, so that it is easy for a reader to find your answers to each question.

# The Data
In this lab, we will use medical data to predict the likelihood of a person experiencing an exercise-induced heart attack.  

Our dataset consists of clinical data from patients who entered the hospital complaining of chest pain ("angina") during exercise. The information collected includes:  

- **age**: Age of the patient
- **sex**: Sex of the patient
- **cp**: Chest Pain type
  - Value 0: asymptomatic
  - Value 1: typical angina
  - Value 2: atypical angina
  - Value 3: non-anginal pain
- **trtbps**: Resting blood pressure (in mm Hg)
- **chol**: Cholesterol in mg/dl fetched via BMI sensor
- **restecg**: Resting electrocardiographic results
  - Value 0: normal
  - Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
  - Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
- **thalach**: Maximum heart rate achieved during exercise
- **output**: The doctor’s diagnosis of whether the patient is at risk for a heart attack
  - 0 = not at risk of heart attack
  - 1 = at risk of heart attack

Although it is not a formal question on this assignment, you should begin by reading in the dataset and briefly exploring and summarizing the data, and by adjusting any variables that need cleaning.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix, roc_curve, cohen_kappa_score
import matplotlib.pyplot as plt
from plotnine import *
from statsmodels.api import Logit

In [ ]:
ha = pd.read_csv("https://www.dropbox.com/s/aohbr6yb9ifmc8w/heart_attack.csv?dl=1")
ha.head()

# Part One: Fitting Models
This section asks you to create a final best model for each of the model types studied this week. For each, you should:

- Find the best model based on ROC AUC for predicting the target variable.
- Report the (cross-validated!) ROC AUC metric.
- Fit the final model.
- Output a confusion matrix; that is, the counts of how many observations fell into each predicted class for each true class.
- (Where applicable) Interpret the coefficients and/or estimates produced by the model fit.

You should certainly try multiple model pipelines to find the best model. You do not need to include the output for every attempted model, but you should describe all of the models explored. You should include any hyperparameter tuning steps in your writeup as well.

## Q1: KNN

## Q2: Logistic Regression

## Q3: Decision Trees

## Q4: Interpretation  
Which predictors were most important to predicting heart attack risk?

## Q5: ROC Curve  
Plot the ROC Curve for your three models above.

# Part Two: Metrics

Consider the following metrics:  

- **True Positive Rate** or **Recall** or **Sensitivity**  
  Of the observations that are truly Class A, how many were predicted to be Class A?  

- **Precision** or **Positive Predictive Value**  
  Of all the observations classified as Class A, how many of them were truly from Class A?  

- **True Negative Rate** or **Specificity** or **Negative Predictive Value**  
  Of all the observations classified as NOT Class A, how many were truly NOT Class A?

Compute each of these metrics (cross-validated) for your three models (KNN, Logistic Regression, and Decision Tree) in Part One.

# Part Three: Discussion  
Suppose you have been hired by a hospital to create classification models for heart attack risk.  

The following questions give a possible scenario for why the hospital is interested in these models. For each one, discuss:  

- Which metric(s) you would use for model selection and why.  
- Which of your final models (Part One Q1-3) you would recommend to the hospital, and why.  
- What score you should expect for your chosen metric(s) using your chosen model to predict future observations.  

## Q1  
The hospital faces severe lawsuits if they deem a patient to be low risk, and that patient later experiences a heart attack.

## Q2  
The hospital is overfull, and wants to only use bed space for patients most in need of monitoring due to heart attack risk.

## Q3  
The hospital is studying root causes of heart attacks, and would like to understand which biological measures are associated with heart attack risk.

## Q4  
The hospital is training a new batch of doctors, and they would like to compare the diagnoses of these doctors to the predictions given by the algorithm to measure the ability of new doctors to diagnose patients.

# Part Four: Validation   

Before sharing the dataset with you, I set aside a random 10% of the observations to serve as a final validation set.  

Use each of your final models in Part One Q1-3, predict the target variable in the validation dataset.  

For each, output a confusion matrix, and report the ROC AUC, the precision, and the recall.  

Compare these values to the cross-validated estimates you reported in Part One and Part Two. Did our measure of model success turn out to be approximately correct for the validation data?  

In [ ]:
ha_validation = pd.read_csv("https://www.dropbox.com/s/jkwqdiyx6o6oad0/heart_attack_validation.csv?dl=1")

# Part Five: Cohen’s Kappa  

Another common metric used in classification is Cohen’s Kappa.  

Use online resources to research this measurement. Calculate it for the models from Part One, Q1-3, and discuss reasons or scenarios that would make us prefer to use this metric as our measure of model success. Do your conclusions from above change if you judge your models using Cohen’s Kappa instead? Does this make sense?  